In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
# genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
# model = genai.GenerativeModel(model_name = "gemini-pro")
#  prompt = [
#     "Write a Python function and explain it to me",
# ]

# response = model.generate_content(prompt)

In [3]:
# print(response.text)

In [4]:
# import google.generativeai as genai
from langchain.llms import GooglePalm
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import SemanticSimilarityExampleSelector

In [5]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt
from langchain.prompts.prompt import PromptTemplate
import ipywidgets as widgets
from IPython.display import display

In [14]:
#fewShot learning 

fewShots= [
    {
     'Question' :"which state has the longest patient stay if duration of stay is the difference between end date and start date?",
     'SQLQuery' :"SELECT State, MAX(DATEDIFF(`EndDate`, `StartDate`)) AS LongestStayDuration FROM hospital_data GROUP BY State ORDER BY LongestStayDuration DESC LIMIT 5;",
     'SQLResult': "Result of the SQL query",
     'Answer': "IL"
    },
    {
     'Question' : "which state has the highest percentage spend for claims paid between 1-30 days after discharge?",
     'SQLQuery' :"SELECT State, MAX(`StateSpendingPercentage`) AS HighestPercentageSpend FROM hospital_data WHERE `Period` = '1 through 30 days After Discharge from Index Hospital Admission' GROUP BY `State` ORDER BY HighestPercentageSpend DESC LIMIT 1;",
     'SQLResult' :"Result of the SQL query",
     'Answer': "FL"
    },
    {
     'Question':"which was the busiest month (had the most number of patients) for claims based on start and end dates?",
     'SQLQuery':"SELECT EXTRACT(MONTH FROM `StartDate`) AS Month,EXTRACT(YEAR FROM `StartDate`) AS Year,COUNT(*) AS ClaimCount FROM hospital_data WHERE `StartDate` IS NOT NULL AND `EndDate` IS NOT NULL GROUP BY Month, Year ORDER BY `ClaimCount` DESC LIMIT 1;",
     'SQLResult':"Result of the SQL query",
     'Answer': "7, 2021"
    },  
    {
     'Question' : "what is max discount received for any patient when comparing invoiced amount to the national average",
     'SQLQuery' :"SELECT (hospital_data.`NationalAvgSpend` - patient_data.`InvoiceAmount`) as Discount FROM patient_data LEFT JOIN hospital_data ON `patient_data.HospitalID` = hospital_data.`ProviderNo` ORDER BY `Discount` LIMIT 1;",
     'SQLResult' :"Result of the SQL query",
     'Answer': "4131"
    },
    {
     'Question' : "what is the most common claim for drug or alcohol users from Colorado",
     'SQLQuery' :"SELECT patient_data.`DrugsAlc`, hospital_data.`ClaimType`, COUNT(*) AS ClaimCount FROM patient_data JOIN hospital_data ON patient_data.`HospitalID` = hospital_data.`ProviderNo WHERE patient_data.`DrugsAlc` = 1 GROUP BY patient_data.`DrugsAlc`, hospital_data.`ClaimType` ORDER BY `ClaimCount` DESC LIMIT 1",
     'SQLResult' :"Result of the SQL query",
     'Answer': "Inpatient (13)"
    },
    {
     'Question' : "What was the largest discount when compared to state average and provided negative feedback?",
     'SQLQuery' :"SELECT patient_data.`PatientID`, patient_data.`InvoiceAmount`, hospital_data.`StateAvgSpend`, patient_data.`Feedback`,(hospital_data.`StateAvgSpend` - patient_data.`InvoiceAmount`) AS Discount FROM patient_data JOIN hospital_data ON patient_data.`HospitalID` = hospital_data.`ProviderNo` WHERE patient_data.`Feedback` = -1 ORDER BY Discount DESC LIMIT 1",
     'SQLResult' :"Result of the SQL query",
     'Answer': "1252.37"
    },
    {
     'Question' : "how many patients paid more than the average hospital amount during a complete episode",
     'SQLQuery' :"SELECT COUNT(*) AS NumPatientsPaidMoreThanAvg FROM patient_data JOIN hospital_data ON patient_data.`HospitalID` = hospital_data.`ProviderNo` WHERE patient_data.`InvoiceAmount` > hospital_data.`HospitalAvgSpend`   AND hospital_data.`Period` = 'Complete Episode' LIMIT 0, 1000",
     'SQLResult' :"Result of the SQL query",
     'Answer': "1"
    },
    {
     'Question' : "Top 3 hospitals which charged an invoice amount from their patients lower than the national average for outpatient?",
     'SQLQuery' :"SELECT hospital_data.`HospitalName`,patient_data.`InvoiceAmount`, hospital_data.`NationalAvgSpend` FROM hospital_data JOIN patient_data ON patient_data.`HospitalID` = hospital_data.`ProviderNo` WHERE hospital_data.`ClaimType` = 'Outpatient'   AND patient_data.`InvoiceAmount` < hospital_data.`NationalAvgSpend` LIMIT 0, 1000",
     'SQLResult' :"Result of the SQL query",
     'Answer': """[('EDEN MEDICAL CENTER', 665.022455860342, 730.0), ('MARTIN GENERAL HOSPITAL', 34.4408549900072, 124.0), ('SAINT FRANCIS BARTLETT MEDICAL CENTER', 52.8388648218745, 124.0)"""
    },
    {
     'Question' : "how many patients are smokers or pregnant from Texas? How many hospitals in the same?",
     'SQLQuery' :"SELECT (SELECT COUNT(*) FROM patient_data WHERE (`Smoker` = 1 OR `Pregnant` = 1) AND `HomeState` = 'TX') AS NumPatients, (SELECT COUNT(*) FROM hospital_data WHERE `State` = 'TX') AS NumHospitals LIMIT 0, 100",
     'SQLResult' :"Result of the SQL query",
     'Answer': "7, 14"
    },
    {
     'Question' : "how many patients with a smoking habit and went to hospitals in their home state during index admission?",
     'SQLQuery' :"SELECT COUNT(*) AS NumSmokerOutpatientPatientsInHomeState FROM patient_data JOIN hospital_data ON patient_data.`HospitalID` = hospital_data.`ProviderNo` WHERE patient_data.`Smoker` = 1   AND patient_data.`HomeState` = hospital_data.`State`   AND hospital_data.`Period` = 'During Index Hospital Admission' LIMIT 0, 100",
     'SQLResult':"Result of the SQL query",
     'Answer': "2"   
    },
    {
     'Question' :"find the number of unemployed patients got treated outside their home state, and ended up getting invoiced for an amount 50% higher than the state average",
     'SQLQuery' :"SELECT COUNT(*) AS NumPatients FROM patient_data JOIN hospital_data ON patient_data.`HospitalID` = hospital_data.`ProviderNo` WHERE patient_data.`Employed` = 0   AND patient_data.`HomeState` != hospital_data.`State`   AND patient_data.`InvoiceAmount` > 1.5 * hospital_data.`StateAvgSpend` LIMIT 0, 1000",
     'SQLResult':"Result of the SQL query",
     'Answer': "3"
    },
    {
     'Question' :"which state had the most number of Drug/Alcohol users as patients",
     'SQLQuery' :"SELECT hospital_data.`State`, COUNT(*) AS NumDrugAlcUsers FROM patient_data JOIN hospital_data ON patient_data.`HospitalID` = hospital_data.`ProviderNo` WHERE patient_data.`DrugsAlc` = 1 GROUP BY hospital_data.`State` ORDER BY NumDrugAlcUsers DESC LIMIT 1",
     'SQLResult':"Result of the SQL query",
     'Answer': "TX (7)"  
    },
    {
     'Question' :"what is the average BMI for outpatients in 2022",
     'SQLQuery' :"SELECT AVG(patient_data.`BMI`) AS AvgBMI FROM patient_data JOIN hospital_data ON patient_data.`HospitalID` = hospital_data.`ProviderNo` WHERE hospital_data.`ClaimType` = 'Outpatient'   AND YEAR(hospital_data.`StartDate`) = 2022 LIMIT 0, 1000",
     'SQLResult':"Result of the SQL query",
     'Answer': "27.86" 
        
    }
   
]

In [9]:
def getfewShot():
#Testing the Database connection
    db_user = "root"
    db_password = "VA9999"
    db_host = "localhost"
    db_name = "hospital"

    #importing the MySQL db 
    db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)
    llm = GooglePalm(google_api_key=os.environ["GOOGLE_API_KEY"], temperature=0.1)
    
    #Sentence transformer models from https://huggingface.co/sentence-transformers?sort_models=modified#models 
    #Using chromaDB for vector storage
    
    x = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2')
    v = [" ".join(i.values()) for i in fewShots]
    vectorstore = Chroma.from_texts(v, x, metadatas=fewShots)
    example_selector = SemanticSimilarityExampleSelector(vectorstore=vectorstore,k=2,)
    
    #from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt
    #from langchain.prompts.prompt import PromptTemplate

    mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
    Pay attention to use only the column names you can see in the tables below, and make sure to reference the right columns from the right tables. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table and use joins to find synergies between tables.
    Pay attention to use CURDATE() function to get the current date, if the question involves "today".
    
    Use the following format:
    
    Question: Question here
    SQLQuery: Query to run with no pre-amble
    SQLResult: Result of the SQLQuery
    Answer: Final answer here
    
    No pre-amble."""
    
    
    example_prompt = PromptTemplate(
        input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
        template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",)
    
    #Creating a prompt 
    
    fewShot_prompt = FewShotPromptTemplate(
        example_selector=example_selector,
        example_prompt=example_prompt,
        prefix=mysql_prompt,
        suffix=PROMPT_SUFFIX,
        input_variables=["input", "table_info", "top_k"],)

    chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=fewShot_prompt)
    return chain

In [10]:
chain = getfewShot()
response = chain.invoke("what is the most common claim for smokers in florida?")

C:\Users\vamsi\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"




> Entering new SQLDatabaseChain chain...
what is the most common claim for smokers in florida?
SQLQuery:

IndexError: list index out of range

In [12]:
def runQuery(question):
    
    chain = getfewShot()
    response = chain.invoke(question)
    return response

def OnButtonClick(b):
    question = question_input.value
    if question:
        answer_output.value = f"Response: {runQuery(question)}"
    else:
        answer_output.value = "Please enter a question."

# Widgets
question_input = widgets.Text(description="Question:")
submit_button = widgets.Button(description="Get Answer")
answer_output = widgets.HTML(value="")

# Event handling
submit_button.on_click(OnButtonClick)


layout = widgets.VBox([question_input, submit_button, answer_output])


In [16]:
display(layout)



> Entering new SQLDatabaseChain chain...
Top 3 hospitals which charged an invoice amount from their patients lower than the national average for outpatient?
SQLQuery:SELECT hospital_data.`HospitalName`,patient_data.`InvoiceAmount`, hospital_data.`NationalAvgSpend` FROM hospital_data JOIN patient_data ON patient_data.`HospitalID` = hospital_data.`ProviderNo` WHERE hospital_data.`ClaimType` = 'Outpatient'   AND patient_data.`InvoiceAmount` < hospital_data.`NationalAvgSpend` LIMIT 0, 1000
SQLResult: [('EDEN MEDICAL CENTER', 665.022455860342, 730.0), ('MARTIN GENERAL HOSPITAL', 34.4408549900072, 124.0), ('SAINT FRANCIS BARTLETT MEDICAL CENTER', 52.8388648218745, 124.0), ("ST MARY'S HOSPITAL ( TROY )", 34.4764545613014, 124.0)]
Answer:['EDEN MEDICAL CENTER', 'MARTIN GENERAL HOSPITAL', 'SAINT FRANCIS BARTLETT MEDICAL CENTER', 'ST MARY'S HOSPITAL ( TROY )']
> Finished chain.


> Entering new SQLDatabaseChain chain...
which illinois hospital charged the highest invoice amount for patients ov